# データの収集
## 取得URLのパース、および映画の評価点数の関数定義

In [1]:
import time
import requests

from bs4 import BeautifulSoup

## ページのソースを表示
def parse_url(url, sleep_second=1):
    res = requests.get(url)
    time.sleep(sleep_second)
    return BeautifulSoup(res.content, "html.parser")

parse_url("https://coco.to/movies")

## 収集する映画の足切り関数
def should_crawl(score, upper_score=90, lower_score=10):
    return lower_score <= int(score.replace("％","")) <= upper_score

## 条件を満たす作品のURLを取得

In [2]:
## 収集先のURLを構築するための変数
domain = "https://coco.to/"
prefix = "movies/%s"
current_year = 2023
n = 15

## 収集の実行
checked_urls = list()
for i in range(n):
    ## 20XX 年の映画一覧ページを取得
    movie_list_url = domain + prefix % (current_year - i)
    soup = parse_url(movie_list_url)
    print("抽出開始", movie_list_url)
    
    # ページからスコアで足切りした作品URLのみを抽出
    # ここの作品情報は <div class=li_panel> 中にあり、その中でスコアは <div class=li_txt>中にある
    # find_all() で作品ごとのパネルのリストを抽出
    for panel in soup.find_all("div", class_="li_panel"):
        # ここの作品(panel)中のスコアを見る
        score = panel.find("div", class_="li_txt").text
        if should_crawl(score):
            # スコアが条件に適している場合はそのURIを href タグから取得し追加
            uri = panel.find("a")["href"]
            checked_urls.append(domain + uri)
    print("抽出終了", movie_list_url, "取得済みURL", len(checked_urls))
print("先頭の1つを表示", checked_urls[0])

抽出開始 https://coco.to/movies/2023
抽出終了 https://coco.to/movies/2023 取得済みURL 8
抽出開始 https://coco.to/movies/2022
抽出終了 https://coco.to/movies/2022 取得済みURL 34
抽出開始 https://coco.to/movies/2021
抽出終了 https://coco.to/movies/2021 取得済みURL 69
抽出開始 https://coco.to/movies/2020
抽出終了 https://coco.to/movies/2020 取得済みURL 114
抽出開始 https://coco.to/movies/2019
抽出終了 https://coco.to/movies/2019 取得済みURL 162
抽出開始 https://coco.to/movies/2018
抽出終了 https://coco.to/movies/2018 取得済みURL 203
抽出開始 https://coco.to/movies/2017
抽出終了 https://coco.to/movies/2017 取得済みURL 252
抽出開始 https://coco.to/movies/2016
抽出終了 https://coco.to/movies/2016 取得済みURL 306
抽出開始 https://coco.to/movies/2015
抽出終了 https://coco.to/movies/2015 取得済みURL 359
抽出開始 https://coco.to/movies/2014
抽出終了 https://coco.to/movies/2014 取得済みURL 398
抽出開始 https://coco.to/movies/2013
抽出終了 https://coco.to/movies/2013 取得済みURL 436
抽出開始 https://coco.to/movies/2012
抽出終了 https://coco.to/movies/2012 取得済みURL 480
抽出開始 https://coco.to/movies/2011
抽出終了 https://coco.to/movies/2011 取得

## 作品ごとのレビューを取得

In [3]:
## プログレスバーの表示
from tqdm import tqdm

crawled = list()
for url in tqdm(checked_urls):
    ## データの取得
    soup = parse_url(url)
    ## レビューコメント
    texts = soup.find_all("div", class_="tweet_text clearflt clearboth")
    ## レビュー評価
    labels = soup.find_all("span", class_="judge_text")
    ## 映画のタイトル
    title = soup.find("div", class_="title_").find("h1")
    ## データのパース
    for t, l in zip(texts, labels):
        text = t.get_text(strip=True)
        metadata = t.find("span").get_text(strip=True)
        stripped_text = text.replace(metadata, "").strip()
        article = {
            "text": stripped_text,
            "label": l.get_text(strip=True),
            "title": title.get_text(strip=True)
        }
        crawled.append(article)
print("全URLの取得完了")

100%|██████████| 588/588 [12:37<00:00,  1.29s/it]

全URLの取得完了


# データの整形

In [63]:
import pandas as pd

df = pd.DataFrame(crawled)

## カラムごとに150文字まで表示
pd.set_option('display.max_colwidth', 150)
# ポジネガの教師データに使うので良いと残念だけに絞る
filtered_by_label = df.query("label == '良い' | label == '残念'")
# ラベルでグルーピングして、各ラベルの数を調べる
group_by_label = filtered_by_label.groupby("label")
labels_size = group_by_label.size()
display(labels_size)
n = labels_size.min()
dataset = group_by_label.apply(lambda x: x.sample(n, random_state=0))
display(dataset)

label
残念    1171
良い    5324
dtype: int64

text  \
label                                                                                                                                             
残念    9494                    『うさぎドロップ』★☆\n\n原作にないわけわからんあの妄想はいらない。ダイキチはそんなやつじゃない。コウキのママのキャラ変更は映画的にうまく脚色されてたと思うが、結局物語はだいぶ変わってしまい脱走の脚色は失敗に終わる・・・。   
      9830                               『SPACE BATTLESHIP ヤマト』いろいろと思うことはあるけど、ギバちゃんは上司役が似合いますね ってかアナライザーが1000機くらいあればガミラス制圧できるんじゃ・・・・#eiga#movie   
      7559                                                                                          『G.I.ジョー　バック2リベンジ』見終わった。なんやこれ…とりあえず目が痛い（苦笑）   
      5730                                                    『96時間　レクイエム』 15点\r\nカット割りが多すぎて何も見えない…\r\n特にチェイスシーンが酷い…\r\n大好きな96時間シリーズの最後がコレじゃ残念。   
      6553                                『17歳』iTunes鑑賞。退屈だった。こういうケースも有るよなぁとは思うけど、だからなんだって思う。度々流れる劇中歌があざとくてうざい。フランス映画らしいわざと小難しく描いた作品。問題作を狙ったのか。   
...                                                                                                                                         ...   
良い    5571  『1001グラム』測量ビジネスをキッチリこなすマリエのお仕事映画。端正でスクウェアな寒色の画面と無表情な彼女も素敵だが、暖色と笑顔の世界に移行するのが、話は定番ではあっても映画の楽しさに満ちている。終盤の量ったり測ったり、の幾つかのラブリーなシーンも好き。#映画   
      8230                                                                                                       『Black & White ブラック＆ホワイト』★★★☆☆   
      6842                 『GODZILLA　ゴジラ』ゴジラが出てくるまでの人間ドラマもいいんだけど、ハリウッド行ってもこのゴジラの神々しさ、自然系の頂点という存在感、否応なしに畏怖する力の表現が良かった。モンスターズの監督だから卵の描写アレだったのかと邪推   
      5857                          『at Home アットホーム』観てきた。泥棒と訳有りの一人者が偽物の家族を演じる。本物の家族愛に恵まれない分…自分で選んだ家族は結び付きが強いのかも…。家族は他人の始まりで血じゃなくて人間の結び付きを見た。#映画   
      6677                    『6才のボクが、大人になるまで。』#wowow録画鑑賞。最後のお母さんの叫びが衝撃。人生って何なんだ？って思ってしまう。長尺ですが、長さを感じさせない余韻の残る良い物語でした。子供達の成長がリアルなので感情移入してしまいます。   

           label                  title  
label                                    
残念    9494    残念                うさぎドロップ  
      9830    残念   SPACE BATTLESHIP ヤマト  
      7559    残念       G.I.ジョー バック2リベンジ  
      5730    残念             96時間　レクイエム  
      6553    残念                    17歳  
...          ...                    ...  
良い    5571    良い    1001グラム　ハカリしれない愛のこと  
      8230    良い  Black＆White／ブラック＆ホワイト  
      6842    良い           GODZILLA　ゴジラ  
      5857    良い         at Home アットホーム  
      6677    良い        6才のボクが、大人になるまで。  

[2342 rows x 3 columns]

In [64]:
from sklearn.preprocessing import LabelEncoder

label_vectorizer = LabelEncoder()

# 残念を0に、良いを1にエンコード
transformed_label = label_vectorizer.fit_transform(dataset.get("label"))
dataset["label"] = transformed_label
display(dataset)

text  \
label                                                                                                                                             
残念    9494                    『うさぎドロップ』★☆\n\n原作にないわけわからんあの妄想はいらない。ダイキチはそんなやつじゃない。コウキのママのキャラ変更は映画的にうまく脚色されてたと思うが、結局物語はだいぶ変わってしまい脱走の脚色は失敗に終わる・・・。   
      9830                               『SPACE BATTLESHIP ヤマト』いろいろと思うことはあるけど、ギバちゃんは上司役が似合いますね ってかアナライザーが1000機くらいあればガミラス制圧できるんじゃ・・・・#eiga#movie   
      7559                                                                                          『G.I.ジョー　バック2リベンジ』見終わった。なんやこれ…とりあえず目が痛い（苦笑）   
      5730                                                    『96時間　レクイエム』 15点\r\nカット割りが多すぎて何も見えない…\r\n特にチェイスシーンが酷い…\r\n大好きな96時間シリーズの最後がコレじゃ残念。   
      6553                                『17歳』iTunes鑑賞。退屈だった。こういうケースも有るよなぁとは思うけど、だからなんだって思う。度々流れる劇中歌があざとくてうざい。フランス映画らしいわざと小難しく描いた作品。問題作を狙ったのか。   
...                                                                                                                                         ...   
良い    5571  『1001グラム』測量ビジネスをキッチリこなすマリエのお仕事映画。端正でスクウェアな寒色の画面と無表情な彼女も素敵だが、暖色と笑顔の世界に移行するのが、話は定番ではあっても映画の楽しさに満ちている。終盤の量ったり測ったり、の幾つかのラブリーなシーンも好き。#映画   
      8230                                                                                                       『Black & White ブラック＆ホワイト』★★★☆☆   
      6842                 『GODZILLA　ゴジラ』ゴジラが出てくるまでの人間ドラマもいいんだけど、ハリウッド行ってもこのゴジラの神々しさ、自然系の頂点という存在感、否応なしに畏怖する力の表現が良かった。モンスターズの監督だから卵の描写アレだったのかと邪推   
      5857                          『at Home アットホーム』観てきた。泥棒と訳有りの一人者が偽物の家族を演じる。本物の家族愛に恵まれない分…自分で選んだ家族は結び付きが強いのかも…。家族は他人の始まりで血じゃなくて人間の結び付きを見た。#映画   
      6677                    『6才のボクが、大人になるまで。』#wowow録画鑑賞。最後のお母さんの叫びが衝撃。人生って何なんだ？って思ってしまう。長尺ですが、長さを感じさせない余韻の残る良い物語でした。子供達の成長がリアルなので感情移入してしまいます。   

            label                  title  
label                                     
残念    9494      0                うさぎドロップ  
      9830      0   SPACE BATTLESHIP ヤマト  
      7559      0       G.I.ジョー バック2リベンジ  
      5730      0             96時間　レクイエム  
      6553      0                    17歳  
...           ...                    ...  
良い    5571      1    1001グラム　ハカリしれない愛のこと  
      8230      1  Black＆White／ブラック＆ホワイト  
      6842      1           GODZILLA　ゴジラ  
      5857      1         at Home アットホーム  
      6677      1        6才のボクが、大人になるまで。  

[2342 rows x 3 columns]

# モデルの構築

## データの分割

In [65]:
from sklearn.model_selection import train_test_split

# 入力と出力に分割
# x（入力）= レビューコメント、y（出力）= "label"
x, y = dataset.get("text"), dataset.get("label")

# 学習とテストデータに 9:1 で分割
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, stratify=y, random_state=0)

# それぞれの数が合っているか確認
print([len(c) for c in [X_train, X_test, y_train, y_test]])

[2107, 235, 2107, 235]


## 形態素分析

In [66]:
from janome.tokenizer import Tokenizer
# wakati=True とすると単語に分かれたオブジェクトを生成
tokenizer = Tokenizer(wakati=True)
print(list(tokenizer.tokenize("今日はいい天気")))

['今日', 'は', 'いい', '天気']


## 単語のベクトル化

In [67]:
from sklearn.feature_extraction.text import CountVectorizer

feature_vectorizer = CountVectorizer(binary=True, analyzer=tokenizer.tokenize)

In [68]:
# ロジスティック回帰による学習

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
transformed_X_train = feature_vectorizer.fit_transform(X_train)
classifier.fit(transformed_X_train, y_train)

LogisticRegression()

In [69]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', random_state=None, solver='lbfgs', tol=0.0001, verbose=0,  warm_start=False)

LogisticRegression()

In [70]:
# モデルの評価

from sklearn.metrics import classification_report

vectorized = feature_vectorizer.transform(X_test)
y_pred = classifier.predict(vectorized)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79       118
           1       0.79      0.76      0.78       117

    accuracy                           0.78       235
   macro avg       0.78      0.78      0.78       235
weighted avg       0.78      0.78      0.78       235



# モデルの分析

In [72]:
# モデルのダンプ

from pandas import Series

feature_to_weight = dict()

for w, name in zip(classifier.coef_[0], feature_vectorizer.get_feature_names_out()):
    feature_to_weight[name] = w

se = Series(feature_to_weight)
se.sort_values(ascending=False, inplace=True)
print("Positive or Negative")
print("--Positiveの判定に効いた素性")
print(se[:20])
print("--Negativeの判定に効いた素性")
print(se[-20:])
print("--" * 50)


Positive or Negative
--Positiveの判定に効いた素性
最高      1.415552
楽しめる    1.205472
！       1.079291
なかなか    1.030442
面白かっ    1.022437
『       1.021728
楽しめ     0.967474
心       0.955346
それでも    0.919935
／       0.919154
劇       0.909970
歳       0.902200
成長      0.900281
15      0.883423
人生      0.865521
しっかり    0.837715
あっ      0.834606
ます      0.811947
楽しい     0.800522
涙       0.794806
dtype: float64
--Negativeの判定に効いた素性
内容      -0.807258
うーん     -0.809559
駄作      -0.845492
ストーリー   -0.872287
意味      -0.877757
地球      -0.881983
雑       -0.912417
正直      -0.920028
つまらない   -0.931901
退屈      -0.963352
全然      -0.966759
ダメ      -0.972448
中途半端    -0.974840
肝心      -0.975159
ちゃっ     -1.037558
なかっ     -1.093008
なん      -1.095386
酷い      -1.108735
すぎ      -1.175292
残念      -2.072586
dtype: float64
----------------------------------------------------------------------------------------------------


In [86]:
# 学習・評価の関数化
def validate():
    # 学習
    classifier = LogisticRegression()
    transformed_X_train = feature_vectorizer.fit_transform(X_train)
    classifier.fit(transformed_X_train, y_train)
    
    # 評価
    vectorized = feature_vectorizer.transform(X_test)
    y_pred = classifier.predict(vectorized)
    print(classification_report(y_test, y_pred))
    
    # モデルのダンプ
    feature_to_weight = dict()

    for w, name in zip(classifier.coef_[0], feature_vectorizer.get_feature_names_out()):
        feature_to_weight[name] = w

    se = Series(feature_to_weight)
    se.sort_values(ascending=False, inplace=True)
    print("Positive or Negative")
    print("--Positiveの判定に効いた素性")
    print(se[:20])
    print("--Negativeの判定に効いた素性")
    print(se[-20:])
    print("--" * 50)
    return y_pred

In [87]:
# 前処理と後処理をかける

from janome.analyzer import Analyzer
from janome.charfilter import RegexReplaceCharFilter
from janome.tokenfilter import ExtractAttributeFilter, POSKeepFilter, TokenFilter

# 前処理
char_filters = [
    RegexReplaceCharFilter("^[『「【].*[』」】]", ""),
    RegexReplaceCharFilter("(https?:\/\/[\w\.\-/:\#\?\=\&\;\%\~\+]*)", "")]

# 後処理
token_filters = [
    POSKeepFilter(['名詞', '動詞', '形容詞', '副詞']),
    ExtractAttributeFilter("base_form")]

# Tokenizer の再初期化
tokenizer = Tokenizer()

# 前処理・後処理が追加されたVectorizerに変更
analyzer = Analyzer(char_filters=char_filters, tokenizer=tokenizer, token_filters=token_filters)
feature_vectorizer = CountVectorizer(binary=True, analyzer=analyzer.analyze)

# 再評価
result = validate()

              precision    recall  f1-score   support

           0       0.70      0.78      0.74       118
           1       0.75      0.66      0.70       117

    accuracy                           0.72       235
   macro avg       0.72      0.72      0.72       235
weighted avg       0.72      0.72      0.72       235

Positive or Negative
--Positiveの判定に効いた素性
最高       1.812464
楽しめる     1.341056
楽しい      1.319604
しっかり     1.158956
切ない      1.067005
成長       1.049555
心        1.043711
なかなか     1.032727
かっこいい    1.026850
本当に      1.011806
素敵       0.995940
涙        0.969969
凄い       0.927749
人生       0.891857
強い       0.888697
可愛い      0.886276
撮影       0.879144
結構       0.859830
見事       0.858147
時        0.856997
dtype: float64
--Negativeの判定に効いた素性
ひどい     -0.894446
正直      -0.909081
入り込める   -0.909904
微妙      -0.927814
全く      -0.933896
肝心      -0.956125
退屈      -0.981977
結局      -0.985467
イマイチ    -1.011929
期待      -1.013471
雑       -1.018461
必要      -1.049834
なん      -1.055371
中途半

In [88]:
# 検証用のDataFrame を作成
validate_df = pd.concat([X_test, y_test], axis=1)
validate_df["y_pred"] = result

# 予測とラベルが異なるものを抽出
false_positive = validate_df.query("y_pred == 1 & label == 0")
display(false_positive)

text  \
label                                                                                                                                                       
残念    6305                                       『PAN ネバーランド、夢のはじまり』ルーニーマーラーさんがキレイって事しか印象が残らなかった。。いっその事全編ミュージカルのようにした方が良い作品になった気がした。。話は壮大な分、どうも、ちまちました展開！   
      7284   今日の映画は009ノ1。坂本浩一監督他の舞台挨拶が有る回だったのだけど終電を逃しそうなので悩んだ末本編だけで退場しました。所謂B級ですね。主役のお姉さんは胸も見せず中途半端なエロと結構血がドバドバなグロ。リアリティは低いけど。話もアレですが女性メインの割にアクションは中々でした。   
      4961                                                                                                             『HiGH&LOW THE MOVIE』みる。2回目、応援上映です。   
      4987                           『変態仮面アブノーマル・クライシス』バカげたことを真面目に演じれば面白いという事を全キャストが実践しているのにただ１人ムロツヨシだけ悪ふざけ演技をしていて映画全体を台無しにしてくれる（しかもスベってる）リアル変態安田顕は良い仕事ね。   
      7219                          『Seventh Code』☆☆+．前田敦子の新曲のPVと知らずに黒沢清作品のつもりで観に行ったのが間違いだったかも．肩透かしの展開と前田敦子のネコパンチ並みのアクションに「なんじゃこりゃ」．見るからに「予算足りなさそう」感が溢れてて   
      10633                                  『しんぼる』枯渇した才能に反比例して肥大したエゴだけが、画面から十分に伝わってきて、テレビの前で笑い転げた思い出がまた一つほろ苦く色あせました。エルカンターレさんのカルト映画の方が笑えるっていうのがあまりにも哀れ・・   
      9878                                            『TSUNAMI ツナミ』泣けるディザスタームービー。だがこれがその手の傑作とは思えない。津波シーンはゴジラ出現シーンを待ちわびる感覚と同様。主人公らの行動から、後半（津波到来後）の展開がミエミエ。   
      5653                                『400デイズ』＠ HTC渋谷♪不測の事態に耐えるためのテストでどんどん広がってく疑問と疑念に面白くなってくばかりだったのに、色々置き去りにしてなんかよくわからないまま終わったw オチがあったら最高に面白かったのにね…。。   
      656                              『100日間生きたワニ』＠ MOVIX亀有♪これは合わなかったな…。とにかく間が長くて多いし、映像の動きも少ないから60分とは思えない体感時間の長さ…。YouTubeとかで各エピソードごとに観たくなるようなテンポとお話かなぁ。。   
      9804                                                                                   『SP 野望篇』CGが凄い＝面白いではないし、リアリティがある＝面白いでもないけど、コレは酷い。見ていて恥ずかしくなる。   
      4341   未体験ゾーン『PLANET OF THE SHARKS 鮫の惑星』観た。地表が海に没した近未来で世界を支配するのはサメだった…しかしこれは毎度お馴染みアサイラム作品。弾けたところがあればいいがいつも以上にショボいCGによく判らない展開で萎えさせてくれる。期待以上には違いない(笑)   
      6337                           『Re:LIFE〜リライフ〜』rewriteという原題の方がいいと思う。既視感がずっとある話だった。幸せの教室にも見えるし、ヒューとドリューバリモアの共演作にも見えた。マリサトメイがキラキラしていて素敵だった。でもそれだけかな…☆2   
      8409                                      『HICK ルリ13歳の旅』★☆\n\nえ？そんな話なの？青春ロードムービーじゃなく、ミステリー？クロエ・モレッツにブレイク・ライブリーが出演してなぜ面白くなくできるのか（笑）説教くさいナレーションもいらない。   
      7994                          『009 RE:CYBORG』神山監督は、自らに内在する彼（ネ申 )の声に従ってこの映画を作ったんだろうけど、こまったもんだ。ゼロからセカイや時空間を創造できるアニメーションの世界で、これだけ不自由にしか動けないキャストが不憫だわネ。   
      6309                           『PAN ネバーランド、夢のはじまり』/なかなか入り込めず、耐えてこれからかなと思ったらNirvanaの大合唱。これはキツイ。以降も映像は詰め込み過ぎで音楽は鳴りっ放し。目にも耳にもうるさくて拒絶反応。脚本も嫌。語りつくせぬ違和感。   
      7295                                                                                                                   『100回泣くこと』涙もろい私が一回も泣けなかった^＾;   
      10484                           『フォース・カインド』凡百あるＰＯＶホラーでも一番酷い映画。ミラジョボビッチ演じる博士の本人役の女優と云う解りづらい女優さんの顔が唯一怖い。ドキュメンタリーという設定だから特殊メイクなしであの顔！登場した時映画館がどよめきました。   
      5814                           『アニー』ストンプを取り入れたハードノックライフは想定内だったけど、音楽のノリはいいのに映画のノリが悪いし笑いも全部はずしてて、スタックスのヅラをいじれる場面が何度もあったのに一度も触れずじまい。なによりアニーが可哀そうに見えない。   
      3215                                 『blank13』円盤で見なきゃよかった…。カメ止めの後に見てしまったので、ちょっと負けた感ある…。2部構成で見せてって後半アドリブというウリはあるもののリンクがあんまりうまくいってないし、役者頼みになってる感も否めず。   
      6963                                                                                                                             『LUCY／ルーシー』何が楽しいのか   
      10251                                                                                            『7つの贈り物』これっていいお話なのか。ただの自分大好き自己中心男の、自己満足な迷惑エピソードだろ。   
      8667                          『REC レック3 ジェネシス』どうしてこうなった。ただまあ、ひとつのギャグ系ゾンビ映画としてなら楽しめるかも。今まで違うというか、レンタルする時、パッケージでどんな感じのゾンビ映画に成り下がったか予想出来てしまった人には、苦作かも。   
      7303                           『100回泣くこと』桐谷美玲の泣き顔がかわいくない。なのに泣いてばっかり。難病ものなのに、見ているこっちはちっとも泣けない。"死が二人を分かつまで"という誓いの言葉が叶ったのはそれはそれで幸せなんよ。。。抗がん剤治療の是非を問う!?   
      305                            『KAPPEI カッペイ』不愉快極まる史上最悪な一本。初っ端からの風俗ネタにこの世に存在していいレベルじゃないギャグの酷さと余りにも酷過ぎる展開の数々で観てるこっちすら馬鹿になる。豪華キャストの無駄遣いを貫いた本当に酷い映画だった…   
      2990                                         『3D彼女　リアルガール』いい歳